In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input, Reshape, Add, Activation
from keras.layers import Conv2D, MaxPooling2D, Maximum, AveragePooling2D, Concatenate, LocallyConnected1D
from keras.initializers import he_normal

Using TensorFlow backend.


In [2]:
img_size = 128

In [4]:
model = Sequential()
model.add(BatchNormalization(input_shape=(img_size, img_size, 3)))

model.add(Conv2D(32, (7, 7), strides=2, padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(1024, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(1024, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 64, 32)        4736      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
dropout_8 (Dropout)          (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 31, 31, 64)        51264     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 29, 29, 64)        36928     
_________________________________________________________________
dropout_9 (Dropout)          (None, 29, 29, 64)        0         
__________

In [26]:
def sub_net(prev_layer, conv_out, dense1_out, out_size, dropouts=[0.1,0.25], kernel_size=3, act='sigmoid'):
    _x_ = Conv2D(conv_out, (kernel_size, kernel_size), padding='same', activation='relu')(prev_layer)
    _x_ = Conv2D(conv_out, (kernel_size, kernel_size), activation='relu')(_x_)
    _x_ = MaxPooling2D(pool_size=2)(_x_)

    _x_ = Dropout(dropouts[0])(_x_)
    _x_ = Flatten()(_x_)
    
    _x_ = Dense(dense1_out, activation='relu')(_x_)
    _x_ = BatchNormalization()(_x_)
    _x_ = Dropout(dropouts[1])(_x_)
    _x_ = Dense(out_size, activation=act)(_x_)

    return _x_

def conv_block(prev_layer, conv_out, dropout = 0.25, kernel_size=3):
    _x_ = Conv2D(conv_out, (1, 1), padding='same', activation='relu')(prev_layer)
    _x_ = Conv2D(conv_out, (kernel_size, kernel_size), activation='relu')(_x_)
    _x_ = AveragePooling2D(pool_size=2)(_x_)
    _x_ = Dropout(dropout)(_x_)

    return _x_

def dense_block(prev_layer, dense_out, dropout = 0.25):
    _x_ = Dense(dense_out, activation='relu')(prev_layer)
    _x_ = BatchNormalization()(_x_)
    _x_ = Dropout(dropout)(_x_)

    return _x_

In [29]:
input_img = Input(shape=(img_size, img_size, 3))

x = BatchNormalization()(input_img)

x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

out1 = conv_block(x, 256, dropout=0.25, kernel_size=3)

out1 = conv_block(out1, 512, dropout=0.25, kernel_size=3)
out1 = Flatten()(out1)
out1 = dense_block(out1, 512, dropout=0.5)
out1 = Dense(17, activation='linear')(out1)

channels = []
for i in range(17):
    outi = conv_block(x, 256, dropout=0.25, kernel_size=3)
    outi = conv_block(outi, 256, dropout=0.25, kernel_size=3)
    outi = Flatten()(outi)
    outi = dense_block(outi, 512, dropout=0.25)
    outi = Dense(1, activation='linear')(outi)
    channels.append(outi)
out2 = Concatenate()(channels)
x = Add()([out1, out2])
x = Activation('sigmoid')(x)

model = Model(inputs=input_img, outputs=x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_17 (InputLayer)            (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
batch_normalization_252 (BatchNo (None, 128, 128, 3)   12                                           
____________________________________________________________________________________________________
conv2d_875 (Conv2D)              (None, 128, 128, 32)  896                                          
____________________________________________________________________________________________________
conv2d_876 (Conv2D)              (None, 126, 126, 32)  9248                                         
___________________________________________________________________________________________

In [4]:
input_img = Input(shape=(img_size, img_size, 3))

x = BatchNormalization()(input_img)

x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

out1 = sub_net(x, 512, dense1_out=512, out_size=17, dropouts=[0.25, 0.5], kernel_size=3, act='linear')

channels = []
for i in range(17):
    channels.append(sub_net(x, 256, dense1_out=512, out_size=1, dropouts=[0.2, 0.2], kernel_size=3, act='linear'))
out2 = Concatenate()(channels)
x = Add()([out1, out2])
x = Activation('sigmoid')(x)

model = Model(inputs=input_img, outputs=x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 128, 128, 3)   12                                           
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 128, 128, 32)  896                                          
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 126, 126, 32)  9248                                         
___________________________________________________________________________________________

In [3]:
def cmsc_inception_layer_4(prev_layer, channels):
    tower_1 = Conv2D(channels, (1, 1), padding='same', activation='relu')(prev_layer)
    tower_1 = BatchNormalization()(tower_1)

    tower_2 = Conv2D(channels, (3, 3), padding='same', activation='relu')(prev_layer)
    tower_2 = BatchNormalization()(tower_2)

    tower_3 = Conv2D(channels, (5, 5), padding='same', activation='relu')(prev_layer)
    tower_3 = BatchNormalization()(tower_3)

    tower_4 = Conv2D(channels, (7, 7), padding='same', activation='relu')(prev_layer)
    tower_4 = BatchNormalization()(tower_4)
    
    output = Maximum()([tower_1, tower_2, tower_3, tower_4])
    return output

def cmsc_inception_layer_3(prev_layer, channels):
    tower_1 = Conv2D(channels, (1, 1), padding='same', activation='relu')(prev_layer)
    tower_1 = BatchNormalization()(tower_1)

    tower_2 = Conv2D(channels, (3, 3), padding='same', activation='relu')(prev_layer)
    tower_2 = BatchNormalization()(tower_2)

    tower_3 = Conv2D(channels, (5, 5), padding='same', activation='relu')(prev_layer)
    tower_3 = BatchNormalization()(tower_3)
    
    output = Maximum()([tower_1, tower_2, tower_3])
    return output

def cmsc_inception_layer_2(prev_layer, channels):
    tower_1 = Conv2D(channels, (1, 1), padding='same', activation='relu')(prev_layer)
    tower_1 = BatchNormalization()(tower_1)

    tower_2 = Conv2D(channels, (3, 3), padding='same', activation='relu')(prev_layer)
    tower_2 = BatchNormalization()(tower_2)
    
    output = Maximum()([tower_1, tower_2])
    return output

In [12]:
model = Sequential()
model.add(BatchNormalization(input_shape=(img_size, img_size, 3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(512, (2, 2), padding='same', activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(Conv2D(512, (2, 2), activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(1024, (1, 1), padding='same', activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(Conv2D(1024, (1, 1), activation='relu', kernel_initializer=he_normal(seed=0)))
model.add(AveragePooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(17, activation='sigmoid', kernel_initializer=he_normal(seed=0)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_9 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 61, 61, 64)        36928     
__________

In [10]:
input_img = Input(shape=(img_size, img_size, 3))

x = BatchNormalization()(input_img)

x = cmsc_inception_layer_4(x, 32)
x = cmsc_inception_layer_3(x, 32)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.3)(x)

x = cmsc_inception_layer_3(x, 64)
x = cmsc_inception_layer_2(x, 64)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.3)(x)

x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(17, activation='sigmoid')(x)

model = Model(inputs=input_img, outputs=x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
batch_normalization_30 (BatchNor (None, 128, 128, 3)   12                                           
____________________________________________________________________________________________________
conv2d_35 (Conv2D)               (None, 128, 128, 32)  128                                          
____________________________________________________________________________________________________
conv2d_36 (Conv2D)               (None, 128, 128, 32)  896                                          
___________________________________________________________________________________________

In [45]:
model = Sequential()
model.add(BatchNormalization(input_shape=(img_size,img_size,3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(512, (2, 2), padding='same', activation='relu'))
model.add(Conv2D(512, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_927 (Bat (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_907 (Conv2D)          (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_908 (Conv2D)          (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_123 (MaxPoolin (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_149 (Dropout)        (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_909 (Conv2D)          (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_910 (Conv2D)          (None, 61, 61, 64)        36928     
__________

In [46]:
from keras.applications import InceptionV3, ResNet50, VGG16, VGG19

In [51]:
model = InceptionV3(weights = "imagenet", include_top=False, input_shape = (139, 139, 3))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_33 (InputLayer)            (None, 139, 139, 3)   0                                            
____________________________________________________________________________________________________
conv2d_917 (Conv2D)              (None, 69, 69, 32)    864                                          
____________________________________________________________________________________________________
batch_normalization_929 (BatchNo (None, 69, 69, 32)    96                                           
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 69, 69, 32)    0                                            
___________________________________________________________________________________________

In [ ]:
img_size = 256

In [19]:
model = Sequential()
model.add(BatchNormalization(input_shape=(img_size,img_size,3)))
model.add(Conv2D(32, (7, 7), strides=2, padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
b
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(1024, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(1024, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
#               optimizer='adam',
#               metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_27 (Batc (None, 256, 256, 3)       12        
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 128, 128, 32)      4736      
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_82 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_150 (Conv2D)          (None, 32, 32, 64)        51264     
_________________________________________________________________
conv2d_151 (Conv2D)          (None, 30, 30, 64)        36928     
__________

In [22]:
img_size=128

In [29]:
model = Sequential()
model.add(BatchNormalization(input_shape=(img_size,img_size,3)))
model.add(Conv2D(32, (7, 7), strides=2, padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), strides=2, padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5),  strides=2, padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
b
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(1024, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(1024, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_44 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_244 (Conv2D)          (None, 64, 64, 32)        4736      
_________________________________________________________________
conv2d_245 (Conv2D)          (None, 62, 62, 32)        9248      
_________________________________________________________________
dropout_137 (Dropout)        (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_246 (Conv2D)          (None, 31, 31, 64)        51264     
_________________________________________________________________
conv2d_247 (Conv2D)          (None, 29, 29, 64)        36928     
_________________________________________________________________
dropout_138 (Dropout)        (None, 29, 29, 64)        0         
__________